## Klasifikasi Text Mining menggunakan Naive Bayes

# load drive

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.append('/content/drive/My Drive/Colab Notebooks/AMS2023')

# install library ekphrasis

In [5]:
# !pip install ekphrasis

In [6]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag","allcaps","elongated","repeated",'emphasis','censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

D:\Anaconda\Lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


D:\Anaconda\Lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


# load library

In [8]:
import re
import string 
import pandas as pd
from copy import deepcopy

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# load dataset

In [10]:
df = pd.read_csv('Dataset_Sentimen_Emosi.csv')

In [11]:
df.head()

Tweet  Sentimen  Emosi
0  emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0     -2
1  Cegah mata rantai Covid-19,mari kita dirumah s...       1.0      1
2  aku mohon yaAllah semoga wabah covid-19 menghi...       1.0     -1
3  Pemprov Papua Naikkan Status Jadi Tanggap Daru...       1.0      1
4            Covid belum nyampe prigen mbak hmm hoax       0.0     -2

karena kita hanya ingin fokus di Sentimen analysis, maka kita hanya pakai `Sentimen` sebagai target class nya **class**

In [13]:
df = df.drop(['Emosi'], axis=1)

In [14]:
df.head()

Tweet  Sentimen
0  emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0
1  Cegah mata rantai Covid-19,mari kita dirumah s...       1.0
2  aku mohon yaAllah semoga wabah covid-19 menghi...       1.0
3  Pemprov Papua Naikkan Status Jadi Tanggap Daru...       1.0
4            Covid belum nyampe prigen mbak hmm hoax       0.0

# Preprocessing

## setting fungsi

In [17]:
def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('RT'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

## running preprocessing
logika main disini

In [19]:
final_string = []
for text in df['Tweet'].values:
    EachReviewText = ""
    proc = bersih_data(text)
    proc = remove_rt(proc) 
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_single_char(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

masukkan hasil preprocessing ke kolom `step01`

In [21]:
df["step01"] = final_string

In [22]:
df.head(1)

Tweet  Sentimen  \
0  emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0   

                                              step01  
0  emang lemes bgt ya mulut lu gue yg mau beli be...

cek jumlah dataset

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     905 non-null    object 
 1   Sentimen  904 non-null    float64
 2   step01    905 non-null    object 
dtypes: float64(1), object(2)
memory usage: 21.3+ KB


## hapus record yang kosong

In [26]:
df_hapus = df[~df['step01'].str.contains(" ")]

In [27]:
df_new = df[~df.isin(df_hapus)].dropna()

In [28]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 903 entries, 0 to 904
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     903 non-null    object 
 1   Sentimen  903 non-null    float64
 2   step01    903 non-null    object 
dtypes: float64(1), object(2)
memory usage: 28.2+ KB


## normalisasi kata slang

In [30]:
# token
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pramu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [32]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [33]:
df_new.head(10)

Tweet  Sentimen  \
0  emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0   
1  Cegah mata rantai Covid-19,mari kita dirumah s...       1.0   
2  aku mohon yaAllah semoga wabah covid-19 menghi...       1.0   
3  Pemprov Papua Naikkan Status Jadi Tanggap Daru...       1.0   
4            Covid belum nyampe prigen mbak hmm hoax       0.0   
5  Nyuruh orang pintar, lu aja Togog. Itu kerumun...      -1.0   
6  Pikir2 balik byk mnde plk nk setelkn lepas covid.       0.0   
7  Selamat pagi, hari jum'at. Jum'at keempat di k...       1.0   
8  Hikmah di balik musibah covid-19, smg para pej...       1.0   
9  Cegah covid-19 beserta jajaran Polsek Kuranji ...       1.0   

                                              step01  \
0  emang lemes bgt ya mulut lu gue yg mau beli be...   
1  cegah mata rantai covid number mari kita dirum...   
2  aku mohon yaallah semoga wabah covid number me...   
3  pemprov papua naikkan status jadi tanggap daru...   
4            covid belum nyampe prigen mbak hmm hoax   
5  nyuruh orang pintar lu aja togog itu kerumunan...   
6   pikir2 balik byk mnde plk nk setelkn lepas covid   
7  selamat pagi hari jum at jum at keempat di kal...   
8  hikmah di balik musibah covid number smg para ...   
9  cegah covid number beserta jajaran polsek kura...   

                                              tokens  
0  [emang, lemes, bgt, ya, mulut, lu, gue, yg, ma...  
1  [cegah, mata, rantai, covid, number, mari, kit...  
2  [aku, mohon, yaallah, semoga, wabah, covid, nu...  
3  [pemprov, papua, naikkan, status, jadi, tangga...  
4    [covid, belum, nyampe, prigen, mbak, hmm, hoax]  
5  [nyuruh, orang, pintar, lu, aja, togog, itu, k...  
6  [pikir2, balik, byk, mnde, plk, nk, setelkn, l...  
7  [selamat, pagi, hari, jum, at, jum, at, keempa...  
8  [hikmah, di, balik, musibah, covid, number, sm...  
9  [cegah, covid, number, beserta, jajaran, polse...

In [34]:
normalized_word = pd.read_excel("kamus perbaikan kata.xlsx")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

df_new['tokens_perbaikan'] = df_new['tokens'].apply(normalized_term)

C:\Users\pramu\AppData\Local\Temp\ipykernel_8896\3176908213.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
C:\Users\pramu\AppData\Local\Temp\ipykernel_8896\3176908213.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [35]:
df_new.head()

Tweet  Sentimen  \
0  emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0   
1  Cegah mata rantai Covid-19,mari kita dirumah s...       1.0   
2  aku mohon yaAllah semoga wabah covid-19 menghi...       1.0   
3  Pemprov Papua Naikkan Status Jadi Tanggap Daru...       1.0   
4            Covid belum nyampe prigen mbak hmm hoax       0.0   

                                              step01  \
0  emang lemes bgt ya mulut lu gue yg mau beli be...   
1  cegah mata rantai covid number mari kita dirum...   
2  aku mohon yaallah semoga wabah covid number me...   
3  pemprov papua naikkan status jadi tanggap daru...   
4            covid belum nyampe prigen mbak hmm hoax   

                                              tokens  \
0  [emang, lemes, bgt, ya, mulut, lu, gue, yg, ma...   
1  [cegah, mata, rantai, covid, number, mari, kit...   
2  [aku, mohon, yaallah, semoga, wabah, covid, nu...   
3  [pemprov, papua, naikkan, status, jadi, tangga...   
4    [covid, belum, nyampe, prigen, mbak, hmm, hoax]   

                                    tokens_perbaikan  
0  [memang, lemes, banget, iya, mulut, kamu, saya...  
1  [cegah, mata, rantai, covid, number, mari, kit...  
2  [aku, mohon, yaallah, semoga, wabah, covid, nu...  
3  [pemprov, papua, naikkan, status, jadi, tangga...  
4  [covid, belum, nyampe, prigen, mbak, aduh, boh...

In [36]:
df_new.iloc[0].tokens

['emang',
 'lemes',
 'bgt',
 'ya',
 'mulut',
 'lu',
 'gue',
 'yg',
 'mau',
 'beli',
 'beli',
 'obat',
 'aja',
 'mikirr',
 'drmn',
 'bisa',
 'beli',
 'obat',
 'kalo',
 'ga',
 'kerja',
 'devil']

In [37]:
df_new.iloc[0].tokens_perbaikan

['memang',
 'lemes',
 'banget',
 'iya',
 'mulut',
 'kamu',
 'saya',
 'yang',
 'mau',
 'beli',
 'beli',
 'obat',
 'saja',
 'mikirr',
 'drmn',
 'bisa',
 'beli',
 'obat',
 'kalau',
 'tidak',
 'kerja',
 'devil']

## Stopword - Menggunakan Modul NLTK

In [39]:
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pramu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(text):
  text = [word for word in text if word not in stopword]
  return text

df_new['Stop_removal'] = df_new['tokens_perbaikan'].apply(lambda x: remove_stopwords(x))
df_new

Tweet  Sentimen  \
0    emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0   
1    Cegah mata rantai Covid-19,mari kita dirumah s...       1.0   
2    aku mohon yaAllah semoga wabah covid-19 menghi...       1.0   
3    Pemprov Papua Naikkan Status Jadi Tanggap Daru...       1.0   
4              Covid belum nyampe prigen mbak hmm hoax       0.0   
..                                                 ...       ...   
900  Seluruh negara di dunia mengalami masa sulit k...       1.0   
901  Setelah covid dan skripsi disaster selesai, ma...       1.0   
902  Malam ini!! Projek "BENDA BOLEH BINCANG" 9 mal...       0.0   
903                Pontang - panting di koyak covid 19      -1.0   
904  Masalahnya sekarang isu jangkitan covid. Alaaa...      -1.0   

                                                step01  \
0    emang lemes bgt ya mulut lu gue yg mau beli be...   
1    cegah mata rantai covid number mari kita dirum...   
2    aku mohon yaallah semoga wabah covid number me...   
3    pemprov papua naikkan status jadi tanggap daru...   
4              covid belum nyampe prigen mbak hmm hoax   
..                                                 ...   
900  seluruh negara di dunia mengalami masa sulit k...   
901  setelah covid dan skripsi disaster selesai mau...   
902  malam ini repeated projek allcaps benda boleh ...   
903              pontang panting di koyak covid number   
904  masalahnya sekarang isu jangkitan covid ala el...   

                                                tokens  \
0    [emang, lemes, bgt, ya, mulut, lu, gue, yg, ma...   
1    [cegah, mata, rantai, covid, number, mari, kit...   
2    [aku, mohon, yaallah, semoga, wabah, covid, nu...   
3    [pemprov, papua, naikkan, status, jadi, tangga...   
4      [covid, belum, nyampe, prigen, mbak, hmm, hoax]   
..                                                 ...   
900  [seluruh, negara, di, dunia, mengalami, masa, ...   
901  [setelah, covid, dan, skripsi, disaster, seles...   
902  [malam, ini, repeated, projek, allcaps, benda,...   
903       [pontang, panting, di, koyak, covid, number]   
904  [masalahnya, sekarang, isu, jangkitan, covid, ...   

                                      tokens_perbaikan  \
0    [memang, lemes, banget, iya, mulut, kamu, saya...   
1    [cegah, mata, rantai, covid, number, mari, kit...   
2    [aku, mohon, yaallah, semoga, wabah, covid, nu...   
3    [pemprov, papua, naikkan, status, jadi, tangga...   
4    [covid, belum, nyampe, prigen, mbak, aduh, boh...   
..                                                 ...   
900  [seluruh, negara, di, dunia, mengalami, masa, ...   
901  [setelah, covid, dan, skripsi, disaster, seles...   
902  [malam, ini, repeated, proyek, allcaps, benda,...   
903       [pontang, panting, di, koyak, covid, number]   
904  [masalahnya, sekarang, isu, jangkitan, covid, ...   

                                          Stop_removal  
0    [lemes, banget, iya, mulut, beli, beli, obat, ...  
1    [cegah, mata, rantai, covid, number, mari, dir...  
2    [mohon, yaallah, semoga, wabah, covid, number,...  
3    [pemprov, papua, naikkan, status, tanggap, dar...  
4          [covid, nyampe, prigen, mbak, aduh, bohong]  
..                                                 ...  
900  [negara, dunia, mengalami, sulit, covid, numbe...  
901  [covid, skripsi, disaster, selesai, banget, be...  
902  [malam, repeated, proyek, allcaps, benda, binc...  
903           [pontang, panting, koyak, covid, number]  
904  [isu, jangkitan, covid, ala, elongated, bosan,...  

[903 rows x 6 columns]

In [41]:
df_new.iloc[0].tokens_perbaikan

['memang',
 'lemes',
 'banget',
 'iya',
 'mulut',
 'kamu',
 'saya',
 'yang',
 'mau',
 'beli',
 'beli',
 'obat',
 'saja',
 'mikirr',
 'drmn',
 'bisa',
 'beli',
 'obat',
 'kalau',
 'tidak',
 'kerja',
 'devil']

In [42]:
df_new.iloc[0].Stop_removal

['lemes',
 'banget',
 'iya',
 'mulut',
 'beli',
 'beli',
 'obat',
 'mikirr',
 'drmn',
 'beli',
 'obat',
 'kerja',
 'devil']

In [43]:
i=0
final_string_tokens = []
for text in df_new['Stop_removal'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [44]:
df_new["step02"] = final_string_tokens

In [45]:
df_new.head()

Tweet  Sentimen  \
0  emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0   
1  Cegah mata rantai Covid-19,mari kita dirumah s...       1.0   
2  aku mohon yaAllah semoga wabah covid-19 menghi...       1.0   
3  Pemprov Papua Naikkan Status Jadi Tanggap Daru...       1.0   
4            Covid belum nyampe prigen mbak hmm hoax       0.0   

                                              step01  \
0  emang lemes bgt ya mulut lu gue yg mau beli be...   
1  cegah mata rantai covid number mari kita dirum...   
2  aku mohon yaallah semoga wabah covid number me...   
3  pemprov papua naikkan status jadi tanggap daru...   
4            covid belum nyampe prigen mbak hmm hoax   

                                              tokens  \
0  [emang, lemes, bgt, ya, mulut, lu, gue, yg, ma...   
1  [cegah, mata, rantai, covid, number, mari, kit...   
2  [aku, mohon, yaallah, semoga, wabah, covid, nu...   
3  [pemprov, papua, naikkan, status, jadi, tangga...   
4    [covid, belum, nyampe, prigen, mbak, hmm, hoax]   

                                    tokens_perbaikan  \
0  [memang, lemes, banget, iya, mulut, kamu, saya...   
1  [cegah, mata, rantai, covid, number, mari, kit...   
2  [aku, mohon, yaallah, semoga, wabah, covid, nu...   
3  [pemprov, papua, naikkan, status, jadi, tangga...   
4  [covid, belum, nyampe, prigen, mbak, aduh, boh...   

                                        Stop_removal  \
0  [lemes, banget, iya, mulut, beli, beli, obat, ...   
1  [cegah, mata, rantai, covid, number, mari, dir...   
2  [mohon, yaallah, semoga, wabah, covid, number,...   
3  [pemprov, papua, naikkan, status, tanggap, dar...   
4        [covid, nyampe, prigen, mbak, aduh, bohong]   

                                              step02  
0  lemes banget iya mulut beli beli obat mikirr d...  
1  cegah mata rantai covid number mari dirumah mi...  
2  mohon yaallah semoga wabah covid number menghi...  
3  pemprov papua naikkan status tanggap darurat c...  
4               covid nyampe prigen mbak aduh bohong

## Stemming - Menggunakan Sastrawi

In [47]:
# !pip install sastrawi

In [48]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [49]:
final_string = []
s = ""
for sentence in df_new["step02"].values:
    filteredSentence = []
    EachReviewText = ""
    s = (stemmer.stem(sentence))
    filteredSentence.append(s)
    
    EachReviewText = ' '.join(filteredSentence)
    final_string.append(EachReviewText)

In [50]:
df_new["ProcessedText"] = final_string

In [51]:
df_new.head(10)

Tweet  Sentimen  \
0  emang lemes bgt ya mulut lu, gue yg mau beli-b...      -1.0   
1  Cegah mata rantai Covid-19,mari kita dirumah s...       1.0   
2  aku mohon yaAllah semoga wabah covid-19 menghi...       1.0   
3  Pemprov Papua Naikkan Status Jadi Tanggap Daru...       1.0   
4            Covid belum nyampe prigen mbak hmm hoax       0.0   
5  Nyuruh orang pintar, lu aja Togog. Itu kerumun...      -1.0   
6  Pikir2 balik byk mnde plk nk setelkn lepas covid.       0.0   
7  Selamat pagi, hari jum'at. Jum'at keempat di k...       1.0   
8  Hikmah di balik musibah covid-19, smg para pej...       1.0   
9  Cegah covid-19 beserta jajaran Polsek Kuranji ...       1.0   

                                              step01  \
0  emang lemes bgt ya mulut lu gue yg mau beli be...   
1  cegah mata rantai covid number mari kita dirum...   
2  aku mohon yaallah semoga wabah covid number me...   
3  pemprov papua naikkan status jadi tanggap daru...   
4            covid belum nyampe prigen mbak hmm hoax   
5  nyuruh orang pintar lu aja togog itu kerumunan...   
6   pikir2 balik byk mnde plk nk setelkn lepas covid   
7  selamat pagi hari jum at jum at keempat di kal...   
8  hikmah di balik musibah covid number smg para ...   
9  cegah covid number beserta jajaran polsek kura...   

                                              tokens  \
0  [emang, lemes, bgt, ya, mulut, lu, gue, yg, ma...   
1  [cegah, mata, rantai, covid, number, mari, kit...   
2  [aku, mohon, yaallah, semoga, wabah, covid, nu...   
3  [pemprov, papua, naikkan, status, jadi, tangga...   
4    [covid, belum, nyampe, prigen, mbak, hmm, hoax]   
5  [nyuruh, orang, pintar, lu, aja, togog, itu, k...   
6  [pikir2, balik, byk, mnde, plk, nk, setelkn, l...   
7  [selamat, pagi, hari, jum, at, jum, at, keempa...   
8  [hikmah, di, balik, musibah, covid, number, sm...   
9  [cegah, covid, number, beserta, jajaran, polse...   

                                    tokens_perbaikan  \
0  [memang, lemes, banget, iya, mulut, kamu, saya...   
1  [cegah, mata, rantai, covid, number, mari, kit...   
2  [aku, mohon, yaallah, semoga, wabah, covid, nu...   
3  [pemprov, papua, naikkan, status, jadi, tangga...   
4  [covid, belum, nyampe, prigen, mbak, aduh, boh...   
5  [nyuruh, orang, pintar, kamu, saja, togog, itu...   
6  [pikir2, balik, banyak, mnde, plk, nk, setelkn...   
7  [selamat, pagi, hari, jum, at, jum, at, keempa...   
8  [hikmah, di, balik, musibah, covid, number, se...   
9  [cegah, covid, number, beserta, jajaran, polse...   

                                        Stop_removal  \
0  [lemes, banget, iya, mulut, beli, beli, obat, ...   
1  [cegah, mata, rantai, covid, number, mari, dir...   
2  [mohon, yaallah, semoga, wabah, covid, number,...   
3  [pemprov, papua, naikkan, status, tanggap, dar...   
4        [covid, nyampe, prigen, mbak, aduh, bohong]   
5  [nyuruh, orang, pintar, togog, kerumunan, oran...   
6     [pikir2, mnde, plk, nk, setelkn, lepas, covid]   
7  [selamat, pagi, jum, at, jum, at, keempat, pan...   
8  [hikmah, musibah, covid, number, semoga, pejab...   
9  [cegah, covid, number, beserta, jajaran, polse...   

                                              step02  \
0  lemes banget iya mulut beli beli obat mikirr d...   
1  cegah mata rantai covid number mari dirumah mi...   
2  mohon yaallah semoga wabah covid number menghi...   
3  pemprov papua naikkan status tanggap darurat c...   
4               covid nyampe prigen mbak aduh bohong   
5  nyuruh orang pintar togog kerumunan orang nula...   
6             pikir2 mnde plk nk setelkn lepas covid   
7  selamat pagi jum at jum at keempat pandemi cov...   
8  hikmah musibah covid number semoga pejabat pem...   
9  cegah covid number beserta jajaran polsek kura...   

                                       ProcessedText  
0  lemes banget iya mulut beli beli obat mikirr d...  
1  cegah mata rantai covid number mari rumah ming...  
2  mohon yaallah moga wabah covid number hilang r...  
3  pemprov papu

## simpan hasil preprocessing

In [53]:
df_new.to_csv('clean_dataset_part01.csv',sep=";")

# Penataan Data

## pembagian fitur dan label

In [56]:
import pandas as pd
import numpy as np

In [57]:
dataset = pd.read_csv('clean_dataset_part01.csv',sep=";")

In [58]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903 entries, 0 to 902
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        903 non-null    int64  
 1   Tweet             903 non-null    object 
 2   Sentimen          903 non-null    float64
 3   step01            903 non-null    object 
 4   tokens            903 non-null    object 
 5   tokens_perbaikan  903 non-null    object 
 6   Stop_removal      903 non-null    object 
 7   step02            903 non-null    object 
 8   ProcessedText     903 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 63.6+ KB


In [59]:
dataset_feature = dataset['ProcessedText'].astype(str)

In [60]:
dataset_feature.head(10)

0    lemes banget iya mulut beli beli obat mikirr d...
1    cegah mata rantai covid number mari rumah ming...
2    mohon yaallah moga wabah covid number hilang r...
3    pemprov papua naik status tanggap darurat covi...
4                 covid nyampe prigen mbak aduh bohong
5    nyuruh orang pintar togog kerumun orang nulari...
6               pikir2 mnde plk nk setelkn lepas covid
7    selamat pagi jum at jum at empat pandemi covid...
8    hikmah musibah covid number moga jabat perinta...
9    cegah covid number serta jajar polsek ranji ak...
Name: ProcessedText, dtype: object

In [61]:
dataset_label = dataset['Sentimen']

In [62]:
dataset_label.head(10)

0   -1.0
1    1.0
2    1.0
3    1.0
4    0.0
5   -1.0
6    0.0
7    1.0
8    1.0
9    1.0
Name: Sentimen, dtype: float64

## distribusi label

In [64]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [65]:
# Visualizing the target variable
plt.figure(figsize=(12,8))
sns.distplot(dataset_label, label=f'target, skew: {dataset_label.skew():.2f}')
plt.legend(loc='best')
plt.show()

C:\Users\pramu\AppData\Local\Temp\ipykernel_8896\1470841120.py:3: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(dataset_label, label=f'target, skew: {dataset_label.skew():.2f}')
D:\Anaconda\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


In [66]:
dataset_label.value_counts()

Sentimen
 1.0    441
 0.0    266
-1.0    196
Name: count, dtype: int64

# Ekstraksi Fitur dan Machine Learning

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

In [69]:
train_x, test_x, train_label, test_label = train_test_split(dataset_feature, dataset_label, test_size = 0.3, random_state=42)

In [70]:
test_x

70     tips ilmuwan tingkat kebal tubuh mari lawan co...
457    cegah covid number serta jajar polsek ranji ak...
218    warga kabupaten lampung utara nyata positif pa...
250    mimpi malam allcaps psbb allcaps tahan batal c...
39     ayo elongated repeated putus mata rantai seman...
                             ...                        
864          orang bahaya covid number lurus belok pergi
638    halo gerak lawan covid number ayo juang inform...
352    personil polsek karanglewas laksana patroli ma...
465    amal langkah langkah cegah tular allcaps covid...
640    wakakakkaka repeated puter cahaya tolong time ...
Name: ProcessedText, Length: 271, dtype: object

## load TF-IDF

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_Vectorizer = TfidfVectorizer()

## Naive Bayes

In [74]:
from sklearn.naive_bayes import MultinomialNB

In [75]:
from sklearn.pipeline import Pipeline
classifier_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('model', MultinomialNB()),
])

In [76]:
parameters_nb = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
    'model__alpha': (0.0001, 0.001, 0.1, 1, 10, 100)
}

In [77]:
classifier_nb = GridSearchCV(classifier_nb, parameters_nb, cv = 3, n_jobs=-1)

In [78]:
classifier_nb.fit(train_x, train_label.ravel())

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'model__alpha': (0.0001, 0.001, 0.1, 1, 10, 100),
                         'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3),
                                                (1, 4)]})

In [79]:
y_pred_nb_train = classifier_nb.predict(train_x)
accuracy_nb_train = accuracy_score(train_label, y_pred_nb_train)
print("Accuracy Training set: ", accuracy_nb_train)

y_pred_nb_test = classifier_nb.predict(test_x)
accuracy_nb_test = accuracy_score(test_label, y_pred_nb_test)
print("Accuracy Test set: ", accuracy_nb_test)

Accuracy Training set:  0.9762658227848101
Accuracy Test set:  0.5904059040590406


In [80]:
recall_nb_train = recall_score(train_label, y_pred_nb_train, average='weighted')
print("Recall Training set: ", recall_nb_train)

recall_nb_test = recall_score(test_label, y_pred_nb_test, average='weighted')
print("Recall Test set: ", recall_nb_test)

Recall Training set:  0.9762658227848101
Recall Test set:  0.5904059040590406


In [81]:
precision_nb_train = precision_score(train_label, y_pred_nb_train, average='weighted')
print("Precision Training set: ", precision_nb_train)

precision_nb_test = precision_score(test_label, y_pred_nb_test, average='weighted')
print("Precision Test set: ", precision_nb_test)

Precision Training set:  0.9763676139089341
Precision Test set:  0.569541371806291


In [82]:
f1_nb_train = f1_score(train_label, y_pred_nb_train, average='weighted')
print("F1 Training set: ", f1_nb_train)

f1_nb_test = f1_score(test_label, y_pred_nb_test, average='weighted')
print("F1 Test set: ", f1_nb_test)

F1 Training set:  0.9762757318902684
F1 Test set:  0.5723164269765864


In [83]:
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(test_label, y_pred_nb_test), annot=True, cmap = 'viridis', fmt='.0f')
plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)
plt.show()

In [84]:
classifier_nb.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('model', MultinomialNB(alpha=0.1))])

In [85]:
for param_name in sorted(parameters_nb.keys()):
    print("%s: %r" % (param_name, classifier_nb.best_params_[param_name]))

model__alpha: 0.1
tfidf__ngram_range: (1, 1)


## KNN

In [87]:
from sklearn.neighbors import KNeighborsClassifier

In [88]:
train_x.dtype

dtype('O')

In [89]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_x_encoded = label_encoder.fit_transform(train_x)

In [90]:
test_x_encoded = label_encoder.fit_transform(test_x)

In [91]:
train_x_encoded.dtype

dtype('int32')

In [92]:
train_x_encoded = train_x_encoded.reshape(-1, 1)
# train_label = train_label.reshape(-1, 1)

In [93]:
test_x_encoded = test_x_encoded.reshape(-1, 1)

In [94]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(train_x_encoded, train_label)

KNeighborsClassifier(n_neighbors=3)

In [95]:
y_pred_knn_train = model.predict(train_x_encoded)
accuracy_knn_train = accuracy_score(train_label, y_pred_knn_train)
print("Accuracy Training set: ", accuracy_knn_train)

y_pred_knn_test = model.predict(test_x_encoded)
accuracy_knn_test = accuracy_score(test_label, y_pred_knn_test)
print("Accuracy Test set: ", accuracy_knn_test)

Accuracy Training set:  0.6819620253164557
Accuracy Test set:  0.36162361623616235


In [96]:
recall_knn_train = recall_score(train_label, y_pred_knn_train, average='weighted')
print("Recall Training set: ", recall_knn_train)

recall_knn_test = recall_score(test_label, y_pred_knn_test, average='weighted')
print("Recall Test set: ", recall_knn_test)

Recall Training set:  0.6819620253164557
Recall Test set:  0.36162361623616235


In [97]:
precision_knn_train = precision_score(train_label, y_pred_knn_train, average='weighted')
print("Precision Training set: ", precision_knn_train)

precision_knn_test = precision_score(test_label, y_pred_knn_test, average='weighted')
print("Precision Test set: ", precision_knn_test)

Precision Training set:  0.7011272070435527
Precision Test set:  0.39319546180536435


In [98]:
f1_knn_train = f1_score(train_label, y_pred_knn_train, average='weighted')
print("F1 Training set: ", f1_knn_train)

f1_knn_test = f1_score(test_label, y_pred_knn_test, average='weighted')
print("F1 Test set: ", f1_knn_test)

F1 Training set:  0.6865068520468985
F1 Test set:  0.3717346622765151


In [99]:
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(test_label, y_pred_knn_test), annot=True, cmap = 'viridis', fmt='.0f')
plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)
plt.show()

# Visualisasi

In [101]:
models = [        
          ('Naive Bayes Multinomial', accuracy_nb_train, accuracy_nb_test),
          ('KNN', accuracy_knn_train, accuracy_knn_test),
         ]

In [102]:
predict = pd.DataFrame(data = models, columns=['Model', 'Training Accuracy', 'Test Accuracy'])
predict

Model  Training Accuracy  Test Accuracy
0  Naive Bayes Multinomial           0.976266       0.590406
1                      KNN           0.681962       0.361624

In [103]:
models_comparison = [                        
                        ('Naive Bayes Multinomial', accuracy_nb_test, recall_nb_test, precision_nb_test, f1_nb_test),
                        ('KNN', accuracy_knn_test, recall_knn_test, precision_knn_test, f1_knn_test),                   
                    ]

In [104]:
comparison = pd.DataFrame(data = models_comparison, columns=['Model', 'Accuracy', 'Recall', 'Precision', 'F1 Weighted'])
comparison

Model  Accuracy    Recall  Precision  F1 Weighted
0  Naive Bayes Multinomial  0.590406  0.590406   0.569541     0.572316
1                      KNN  0.361624  0.361624   0.393195     0.371735

In [105]:
import numpy as np

f, axes = plt.subplots(2,1, figsize=(14,10))

predict.sort_values(by=['Training Accuracy'], ascending=False, inplace=True)

sns.barplot(x='Training Accuracy', y='Model', data = predict, palette='Blues_d', ax = axes[0])
#axes[0].set(xlabel='Region', ylabel='Charges')
axes[0].set_xlabel('Training Accuracy', size=16)
axes[0].set_ylabel('Model')
axes[0].set_xlim(0,1.0)
axes[0].set_xticks(np.arange(0, 1.1, 0.1))

predict.sort_values(by=['Test Accuracy'], ascending=False, inplace=True)

sns.barplot(x='Test Accuracy', y='Model', data = predict, palette='Greens_d', ax = axes[1])
#axes[0].set(xlabel='Region', ylabel='Charges')
axes[1].set_xlabel('Test Accuracy', size=16)
axes[1].set_ylabel('Model')
axes[1].set_xlim(0,1.0)
axes[1].set_xticks(np.arange(0, 1.1, 0.1))

plt.show()

# simpan kedalam joblib

In [107]:
import joblib

In [108]:
# save the model to disk
joblib.dump(classifier_nb.best_estimator_, 'classifier_nb.best_estimator_.sav', compress=1)

['classifier_nb.best_estimator_.sav']